# 网页爬虫

不管是商业数据集，还是财经数据集，很多被嵌入到网页的结构和样式中，难以复制，需要使用特殊方法抓取出来。从网页中抽取数据的过程被称为**网络爬虫**。

## 1. 基本概念

客户端(Clients)通常是浏览器（Chrome、Edge、Safari），它们可以是任何类型的程序或设备。  
服务器(Servers)最常见的是云端服务器。

![clients_server](image/clients_server.png)

客户端(Clinet)和服务器(Server)之间的通信是通过请求(request)和响应(response)完成的：

1. 客户端,通常是浏览器向网络发送一个HTTP请求(request)
2. 服务器收到该请求
3. 服务器运行一个应用程序来处理该请求
4. 服务器向浏览器返回一个HTTP响应（response）
5. 浏览器收到该响应


以下是一些和网络爬虫相关，你需要知道的基本概念：

|名词|英文全称|概念解释|
|---|---|---|
|HTTP|Hypertext Transfer Protocol|超文本传输协议（HTTP）是互联网协议套件模型中的一个应用层协议，用于分布式、协作式、超媒体信息系统|
|URL|Uniform Resource Locator|它是互联网资源的地址，它包括一个协议，一个域名，有时还包括其他定位信息|
|认证|Authentication|它对访问者的身份进行验证，例如验证用户输入的密码是否正确|
|重定向|URL Redirections|它是一种网络技术，使一个网页在多个URL地址下可用|
|Cookies|HTTP Cookies|它是用户浏览网站时在本地存储的相关信息，包括身份信息、登录信息、购物车和历史记录等|


## 2. 使用urllib库抓取网页

urllib作为Python的标准库，基本上涵盖了基础的网络请求功能。

urllib中，request这个模块主要负责构造和发起网络请求，并在其中加入Headers、Proxy等。它还提供了一个稍微复杂的接口来处理常见的情况--如基本认证、cookies、代理等等。

使用 urllib.request 的最简单方法如下：

In [76]:
from urllib import request
resp = request.urlopen('http://www.baidu.com')
content = resp.read().decode('utf-8')   #根据网站对于文字的编码类型，进行解码

输出content的部分，例如前1000个字符：

In [77]:
content[:1000]

'<!DOCTYPE html><!--STATUS OK--><html><head><meta http-equiv="Content-Type" content="text/html;charset=utf-8"><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"><meta content="always" name="referrer"><meta name="theme-color" content="#ffffff"><meta name="description" content="全球领先的中文搜索引擎、致力于让网民更便捷地获取信息，找到所求。百度超过千亿的中文网页数据库，可以瞬间找到相关的搜索结果。"><link rel="shortcut icon" href="/favicon.ico" type="image/x-icon" /><link rel="search" type="application/opensearchdescription+xml" href="/content-search.xml" title="百度搜索" /><link rel="icon" sizes="any" mask href="//www.baidu.com/img/baidu_85beaf5496f291521eb75ba38eacbd87.svg"><link rel="dns-prefetch" href="//dss0.bdstatic.com"/><link rel="dns-prefetch" href="//dss1.bdstatic.com"/><link rel="dns-prefetch" href="//ss1.bdstatic.com"/><link rel="dns-prefetch" href="//sp0.baidu.com"/><link rel="dns-prefetch" href="//sp1.baidu.com"/><link rel="dns-prefetch" href="//sp2.baidu.com"/><link rel="apple-touch-icon-precomposed" href="https://psstatic.cd

上述返回的内容是一个使用HTML语言编写的字符串。HTML是超文本标记语言的缩写，它是网页的标准标记语言。上述内容包含了由<>标签表示的大量HTML元素。

## 3. 使用requests库抓取网页

[requests](https://docs.python-requests.org/en/latest/)是一个流行的爬虫工具，它在urllib3上进行封装，使得爬虫变得更加简单。

我们尝试在JD电商平台上，选择某个商品，例如巧克力。

**第一步，使用requests.get()函数获取html文件**

In [32]:
import requests
headers = { "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:91.0) Gecko/20100101 Firefox/91.0"}
url = 'https://search.jd.com/Search?keyword=%E5%B7%A7%E5%85%8B%E5%8A%9B&enc=utf-8&wq=%E5%B7%A7%E5%85%8B%E5%8A%9B&pvid=2189bd5092fb4ba59b559a5f452ec54e'
req = requests.get(url=url, headers=headers)
html_doc = req.content.decode("utf-8")

**第二步，我们解析图片里面这句话**

<center><img src="image/JD_requests2.png" alt="image/JD_requests2.png" width="600"></center>


上图的`“歌帝梵(GODIVA) 臻粹进口”`在html的XPath路径为`“/html/body/div[5]/div[2]/div[2]/div[1]/div/div[2]/ul/li[2]/div/div[3]/a/em”`。  
lxml是处理XML和HTML的Python库。使用方法见[官网](https://lxml.de/tutorial.html)。

XPath，全称XML Path Language，即XML路径语言，它是一门在XML文档中查找信息的语言，它最初是用来搜寻XML文档的，但是它同样适用于HTML文档的搜索。

In [33]:
from lxml import etree
html_obj = etree.HTML(html_doc)
xpath = '/html/body/div[5]/div[2]/div[2]/div[1]/div/div[2]/ul/li[1]/div/div[3]/a/em'
xpath = xpath + '/text()'     #加了/text()来获取文本内容
context = html_obj.xpath(xpath)[0]
context

'\n脆香米牛奶'

**第三步，我们设计一个函数，用来解析html文件里面的元素**

提取每一条商品的信息，包含商品名称、价格、店铺等信息。

In [37]:
from lxml import etree
def parse_html(html_doc):
    """
    解析html获取数据
    """
    print("parse_html ...")
    html_obj = etree.HTML(html_doc)
    products_dict = {}
    i= 1 
    while True:
        try:
            # 商品名称
            xpath = '/html/body/div[5]/div[2]/div[2]/div[1]/div/div[2]/ul/li[%s]/div/div[3]/a/em' %i
            xpath = xpath + '/text()'     
            title = html_obj.xpath(xpath)[0]
            title = title.split('\n')[1]

            # 商品价格
            xpath = '/html/body/div[5]/div[2]/div[2]/div[1]/div/div[2]/ul/li[%i]/div/div[2]/strong/i' %i
            xpath = xpath + '/text()'     
            price = html_obj.xpath(xpath)[0]

            # 店铺名称
            xpath = '/html/body/div[5]/div[2]/div[2]/div[1]/div/div[2]/ul/li[%i]/div/div[5]/span/a' %i
            xpath = xpath + '/text()'     
            store = html_obj.xpath(xpath)[0]

            products_dict[i] = {"商品名称":title,
                            "价格":price, 
                            "店铺":store}
            i += 1
        except IndexError:
            break
    return products_dict

In [39]:
p = parse_html(html_doc)

parse_html ...


In [41]:
import pandas as pd
pd.DataFrame(p).T   #transpose

,商品名称,价格,店铺
1,脆香米牛奶,24.90,德芙京东自营旗舰店
2,费列罗（FERRERO）榛果威化糖果,139.90,费列罗京东自营旗舰店
3,士力架花生夹心,56.90,德芙京东自营旗舰店
4,健达（Kinder）缤纷乐牛奶榛果,22.70,费列罗京东自营旗舰店
5,德芙（Dove）丝滑牛奶,32.90,德芙京东自营旗舰店
6,德芙（Dove）什锦碗装三种口味混合249g 休闲零食糖果,32.90,德芙京东自营旗舰店
7,德芙（Dove）牛奶,52.90,德芙京东自营旗舰店
8,费列罗（FERRERO）榛果威化,9.90,费列罗京东自营旗舰店
9,Lindt瑞士莲软心 瑞士进口精选,145.00,瑞士莲京东自营旗舰店
10,三角（Toblerone）瑞士牛奶,18.90,三角金象巧克力京东自营专区


### 练习

使用lxml.etree模块和其解析xpath()方法，访问JD网页，爬取某个商品。

## 4.正则表达式的识别

正则表达式(Regular Expression)，又称规则表达式，是构成或分解字符串的规则。它使用单个字符串来描述、匹配一系列符合某个句法规则的字符串。

最常用的方法是`re.search`，其扫描整个字符串并返回第一个成功的匹配。

In [4]:
import re
string = 'www.baidu.comwww'
pattern = 'w{3}$'
loc = re.search(pattern, string).span()
print(loc, string[loc[0]:loc[1]])               

(13, 16) www


正则表达式模式如下：

![re_rules](image/re_rules.png)

### 练习

给定如下字符串，匹配出字符串'abc', '123', '3de'。

In [18]:
string = "abc123def"




## 参考
- HTTP的概念: https://www.w3schools.com/whatis/whatis_http.asp
- Cookies: https://www.kaspersky.com/resource-center/definitions/cookies
- urllib官方文档: https://docs.python.org/3/library/urllib.request.html#  
- HOWTO Fetch Internet Resources Using The urllib Package:https://docs.python.org/3/howto/urllib2.html#urllib-howto